In [ ]:
import pickle

with open('dataframe_recap_sampling_no_sampling.pkl', 'rb') as file:
    dataframe_recap_sampling_no_sampling = pickle.load(file)

with open('best_results_df2.pkl', 'rb') as file:
    best_results_df2 = pickle.load(file)

In [ ]:
latex_code = dataframe_recap_sampling_no_sampling.to_latex(index=False)
latex_code

In [ ]:
latex_code = best_results_df2.to_latex(index=False)
latex_code

In [ ]:
with open('modele_regression_logistique.pkl', 'rb') as file:
    modele_regression_logistique = pickle.load(file)

with open('modele_xgboost.pkl', 'rb') as file:
    modele_xgboost = pickle.load(file)

In [ ]:
with open('modele_adaboost.pkl', 'rb') as file:
    modele_adaboost = pickle.load(file)

with open('modele_arbre_decision.pkl', 'rb') as file:
    modele_arbre_decision = pickle.load(file)

In [ ]:
import pickle

with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

In [ ]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

In [ ]:
train = df.loc[(df['DateTransaction'] >= train_inf) & (df['DateTransaction'] <= train_sup)]
X_train = train.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_train = train['FlagImpaye']

In [ ]:
test = df.loc[(df['DateTransaction'] >= test_inf) & (df['DateTransaction'] <= test_sup)]
X_test = test.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_test = test['FlagImpaye']

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix

# Assume que X et y sont vos features et labels respectivement
# Assurez-vous d'importer vos données correctement avant d'utiliser ce code

# Utilisation du Random Oversampler
oversampler = RandomOverSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

# Initialisation du modèle AdaBoost
adaboost_model = AdaBoostClassifier()

# Entraînement du modèle sur les données échantillonnées
adaboost_model.fit(X_resampled, y_resampled)

# Prédiction sur l'ensemble de test
y_pred = adaboost_model.predict(X_test)

# Calcul du F1 Score
f1 = f1_score(y_test, y_pred)

conf_matrix = confusion_matrix(y_test, y_pred)

print(f"F1 Score du modèle AdaBoost : {f1}")
print("Matrice de confusion :\n", conf_matrix)


In [ ]:
from sklearn.metrics import make_scorer

# fonction de coût spécifique en fonction des résultats du modèle
def custom_cost(y_true, y_pred, dataframe):
    marge = 0
    
    for true, pred, montant in zip(y_true, y_pred, dataframe['Montant']):
        if true == 1 and pred == 0:  # FN
            if montant <= 20:
                marge = marge
            elif montant <= 50:
                marge -= 0.2 * montant
            elif montant <= 100:
                marge -= 0.3 * montant
            elif montant <= 200:
                marge -= 0.5 * montant
            else:
                marge -= 0.8 * montant

        elif true == 0 and pred == 1:  # FP
            marge += 0.7 * 0.05 * montant
        elif true == 0 and pred == 0:  # TN
            marge += 0.05 * montant
        # Le cas TP n'ajoute pas de coût
    
    return marge

marge = custom_cost(y_test, y_pred, X_test)
print("Montant de la prédiction en utilisant la fonction de coût personnalisée :", marge)